In [1]:
from lxml import etree
import glob
import pandas as pd
import numpy as np
from pathlib import Path
import os,sys
from tqdm import tqdm
import re
from PIL import Image

### 接口

In [4]:
def voc2yolo():
    center_x = (xmin + (xmax-xmin)/2)/width
    center_y = (ymin + (ymax-ymin)/2)/height
    bdb_w = (xmax-xmin)/width
    bdb_h = (ymax-ymin)/height

In [43]:
def parse_xml(p):
    trees = etree.parse(str(p))
    width, height = int(trees.find('size/width').text), int(trees.find('size/height').text)
    bboxes = []
    for obj in trees.findall('object'):
        bx = []
        bx.append(p)
        bx.append(int(width))
        bx.append(int(height))
        bdb = obj.find('bndbox')
        bx.append(obj.find('name').text)
        bx.append(int(bdb.find('xmin').text))
        bx.append(int(bdb.find('ymin').text))
        bx.append(int(bdb.find('xmax').text))
        bx.append(int(bdb.find('ymax').text))
        bboxes.append(bx)
    return bboxes

In [6]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}
model_label_map8 = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装_短款
    3: 5,  # 裤子_短款
    4: 6,  # 裙子
    5: 7,  # 连体装
    6: 4,  # 上装_长款
    7: 5,  # 裤子_长款
}

In [7]:
model_label_map9 = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装_短款
    3: 5,  # 裤子_长款
    4: 6,  # 裙子
    5: 7,  # 连体装
    6: 4,  # 上装_长款
    7: 5,  # 超短裤
    8: 4,  # 超短裙
}

id2chi9 = {
    0: '鞋子',
    1: '包包',
    2: '上装',
    3: '裤子',
    4: '半身裙',
    5: '连体装',
    6: '上装-长款',
    7: '超短裤',
    8: '超短裙',
}

id2eng9 = {
    0: 'shoe',  # 鞋子
    1: 'bag',  # 包包
    2: 'upper_short',  # 上装
    3: 'paints',  # 裤子
    4: 'skirt',  # 裙子
    5: 'wholebody',  # 连体装
    6: 'upper_long',
    7: 'paints_hot',
    8: 'miniskirt',
}

### 1.Data Loader.

In [16]:
root = Path('/nas/chenyi/datasets_nas/deploy_system/deploy_manual_sep/')
# !ls $root_xml
!ls $root

images	xml


In [29]:
!ls /nas/chenyi/datasets_nas/deploy_system/deploy_manual_sep/images/apparel/

anomaly0826  modified_catg


In [1]:
p = '/home/chenyi/workspace/tbk-service/tbk_cy/'
import os
os.path.islink(p)

False

In [33]:
root_img = root /'images/apparel/anomaly0826/'
img_list = list(root_img.glob('*.jpg'))
img_list += list(root_img.glob('*.jpeg'))
img_list += list(root_img.glob('*.png'))
img_df = pd.DataFrame()
img_df['path'] = list(img_list)
img_df['stem'] = img_df['path'].apply(lambda x: Path(x).stem)
len(img_list)

1929

In [18]:
# 核查xml是否有重复
root_xml = root /'xml/anomaly0826_xml'
xml_list = root_xml.glob('*.xml')
xml_df = pd.DataFrame()
xml_df['path'] = list(xml_list)
xml_df['stem'] = xml_df['path'].apply(lambda x: Path(x).stem)
len(xml_df)

1929

#### csv读取和保存

In [35]:
csv_fname = 'anomaly0826.csv'
root_csv = Path('/nas/chenyi/datasets_nas/deploy_system/deploy_manual_sep/xml')
csv_p = root_csv /csv_fname
# bboxes_df = pd.read_csv(root_csv /'gall7k_xml_20220715.csv')
# parts_df = pd.read_csv('../../data/xml_csv/gall7k_xml_20220714.csv')

#### !mkdir -p $root_csv
# bboxes_df = bboxes_all_df
# bboxes_df.to_csv(str(csv_p), index=False)

In [ ]:
bboxes_df['label9'] = bboxes_df['labelimg'].map(chi2id9)
bboxes_df['label_eng'] = bboxes_df['label9'].map(id2eng9)

#### 数据分析

In [ ]:
bboxes_df.value_counts('labelimg'), len(bboxes_df), len(bboxes_df.drop_duplicates('stem')), len(bboxes_df.value_counts('labelimg'))

### 解析XML

#### 解析XML

In [40]:
bboxes_list = []
for p in tqdm(xml_df['path']):
    try:
        row = parse_xml(p)
        bboxes_list += row
    except:
        print('empty xml.')
    # break

100%|█████████████████████████████████████████████████████████████████████████████| 1929/1929 [00:01<00:00, 1454.61it/s]


In [41]:
bboxes_df = pd.DataFrame(bboxes_list)
name_stand = ['xml_path', 'width', 'height', 'labelimg', 'bx1', 'bx2','bx3', 'bx4']
bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
                          bboxes_df.columns[1]:name_stand[1],
                          bboxes_df.columns[2]:name_stand[2],
                          bboxes_df.columns[3]:name_stand[3],
                          bboxes_df.columns[4]:name_stand[4],
                          bboxes_df.columns[5]:name_stand[5],
                          bboxes_df.columns[6]:name_stand[6],
                          bboxes_df.columns[7]:name_stand[7],
                         },
                          inplace=True)

In [42]:
bboxes_df.value_counts('labelimg'), len(bboxes_df), len(bboxes_df.value_counts('labelimg').index)

(labelimg
 包包       898
 上装       550
 连体装      231
 裤子       109
 超短裙       65
 鞋子        65
 半身裙       58
 超短裤       50
 上装-长款     13
 包          5
 dtype: int64,
 2044,
 10)

In [17]:
bboxes_df['stem'] = bboxes_df['xml_path'].apply(lambda x: Path(x).stem)

In [23]:
bboxes_df['label96'] = bboxes_df['labelimg'].map(chi2id9)

In [28]:
bboxes_df.value_counts('label96').sort_index()

label96
0    888
1    308
2    199
3      3
4     48
5    418
6     41
7      4
8     29
dtype: int64

##### 修正label

In [18]:
len(bboxes_df.value_counts('labelimg'))

9

In [ ]:
label_name_chi  = 'uuper_long'
sample_df = bboxes_df[bboxes_df['labelimg']==label_name_chi]
# p_sample = sample_df['img_sp'].values[0]

In [ ]:
name_new = 'upper_long'
bboxes_df.loc[bboxes_df[bboxes_df['labelimg']==label_name_chi].index, 'labelimg'] = name_new

In [ ]:
sample_df

#### 获取对应图片信息

In [25]:
# imgs_list = glob.glob(str(root /'images/*/*/*/*'))
img_df = pd.DataFrame()
img_df['path'] = imgs_list
len(img_df)

520

In [27]:
img_df['stem'] = img_df['path'].apply(lambda x: Path(x).stem)

In [ ]:
# img_df['parts3'] = img_df['path'].apply(lambda x: Path(x).parts[-3])

In [ ]:
# img_df.value_counts('parts3')

##### 获取bbox对应的图片

In [29]:
# xml是否都有对应的图片
cross_df = img_df[img_df['stem'].isin(bboxes_df['stem'])]
len(cross_df)

520

In [30]:
stem2path = get_list2dict(cross_df['stem'].values, cross_df['path'].values)

In [31]:
bboxes_df['img_sp'] = bboxes_df['stem'].map(stem2path)

In [32]:
bboxes_clr_df = bboxes_df[~bboxes_df['img_sp'].isna()]

In [33]:
len(bboxes_df), len(bboxes_clr_df)

(1938, 1938)

##### 获取图片的宽和高

In [ ]:
bboxes_df = bboxes_clr_df 

In [ ]:
w_list = []
h_list = []
for ind,row in tqdm(bboxes_df.iterrows()):
    img_sp = row['img_sp']
    try:
        img = Image.open(img_sp)
        w, h = img.size
    except:
        try:
            img = cv2.imread(sample_p)
            w,h = img.shape[1], img.shape[0]
        except:
            w,h = None, None
    w_list.append(w)
    h_list.append(h)
    # break

1938it [00:05, 336.59it/s]


In [36]:
bboxes_df['width'] = w_list
bboxes_df['height'] = h_list

In [37]:
na_df = bboxes_df[bboxes_df['width'].isna()]
na_df

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,label96,img_sp


#### label处理

In [ ]:
# bboxes_df['label9'] = bboxes_df['labelimg'].map(id2eng9)
# bboxes_df['label9'] = bboxes_df['labelimg'].map(eng2id9)

In [38]:
bboxes_df['label_model'] = bboxes_df['label96'].values
bboxes_df.loc[bboxes_df['label_model']==6, 'label_model'] = 2
bboxes_df.loc[bboxes_df['label_model']==7, 'label_model'] = 3
bboxes_df.loc[bboxes_df['label_model']==8, 'label_model'] = 4

##### 统计人员的bbox

In [ ]:
df = bboxes_df
df.tail(1).values

In [ ]:
# df['man'] = df['xml_path'].apply(lambda x: Path(x).parts[5])
df.value_counts('man').sort_index()

### one object.

In [ ]:
eng2num = {'shoe': 0, 
         'upperbody_short':2,
         'bag':1, 
         'lowbody_long':7, 
         'upperbody_long':6,
         'skirt':4, 
         'wholebody':5, 
         'lowbody_short':3}

In [ ]:
eng2num_hard = {'0_shoe': 0, 
                 '1_bag':1, 
         '2_upperbody_short':2,
         '3_lowbody_short':3, 
         '4_skirt':4, 
         '5_wholebody':5, 
         '6_upperbody_long':6,
         '7_lowbody_long':7}

In [ ]:
bboxes_df['cls_folder'] = bboxes_df['img_sp'].apply(lambda x: Path(x).parts[-2])

In [ ]:
# bboxes_df['label_folder'] = bboxes_df['cls_folder'].map(eng2num)
bboxes_df['label_folder'] = bboxes_df['cls_folder'].map(eng2num_hard)

In [ ]:
bboxes_eq_df = bboxes_df[bboxes_df['label_folder']==bboxes_df['label86']]

In [ ]:
bboxes_df['label_query'] = bboxes_df['label_folder']==bboxes_df['label86']

In [ ]:
len(bboxes_eq_df)

### 计算打标准确率

In [ ]:
parts_df = pd.read_csv('/nas/chenyi/datasets_nas/gallery2m_clred/gallery2m_7k20220715/csv/gall7k_xml_20220715.csv')

In [ ]:
parts_df.head(1)

In [ ]:
counter = parts_bboxes_df.value_counts('man').sort_index()
counter

In [ ]:
stem2part = get_list2dict(parts_df['stem'].values, parts_df['man'].values)

In [ ]:
parts_bboxes_df = parts_df[parts_df['stem'].isin(bboxes_df['stem'])]
len(parts_bboxes_df)

In [ ]:
bboxes_df['man'] = bboxes_df['stem'].map(stem2part)

In [ ]:
counter_gt = bboxes_df.value_counts('man').sort_index()
counter_gt

In [ ]:
def get_iou(BBGT, bb):
    # intersection
    ixmin = np.maximum(BBGT[:, 0], bb[0])
    iymin = np.maximum(BBGT[:, 1], bb[1])
    ixmax = np.minimum(BBGT[:, 2], bb[2])
    iymax = np.minimum(BBGT[:, 3], bb[3])
    iw = np.maximum(ixmax - ixmin + 1., 0.)
    ih = np.maximum(iymax - iymin + 1., 0.)
    inters = iw * ih        # union
    uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
           (BBGT[:, 2] - BBGT[:, 0] + 1.) *
           (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

    return inters / uni

In [ ]:
label_df1 = parts_bboxes_df[parts_bboxes_df['man']=='part5_img_xml']
gt_df1 = bboxes_df[bboxes_df['man']=='part5_img_xml']
parts_gp = dict(list(label_df1.groupby('stem')))
bboxes_gp = dict(list(gt_df1.groupby('stem')))

In [ ]:
tp_all, fp_all = 0, 0
stem_list = {}
for k in tqdm(bboxes_gp.keys()):
    item_part = parts_gp[k]
    item_bbox = bboxes_gp[k]
    bbx1 = item_part.loc[:, ['bx1', 'bx2', 'bx3', 'bx4', 'label9']].values
    bbx_gt = item_bbox.loc[:, ['bx1', 'bx2', 'bx3', 'bx4', 'label9']].values

    # bbxes_pred = bbx1
    # BBGT = bbx_gt
    # ovthresh = 0.90
    t, f = eval_one_img(bbx1, bbx_gt, ovthresh = 0.9)
    tp_all += t
    fp_all += f
    stem_list[k] = [t, f]

In [ ]:
# part1
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part2
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part3
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part4
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
# part5
fn_all = len(gt_df1) - tp_all
tp_all / (tp_all + fp_all) , tp_all / (tp_all + fn_all)

In [ ]:
fn_all = len(parts_bboxes_df) - tp_all

In [ ]:
tp_all / (tp_all + fp_all)

In [ ]:
tp_all / (tp_all + fn_all)

In [ ]:
def eval_one_img(bbxes_pred, BBGT, ovthresh = 0.5):
    # 按照置信度降序排序sorted_ind = np.argsort(-confidence)
    # BB = BB[sorted_ind, :]   # 预测框坐标image_ids = [image_ids[x] for x in sorted_ind] # 各个预测框的对应图片id# 便利预测框，并统计TPs和FPsnd = len(image_ids)

    # 获取每个类别的所有bboxes，包括gt和预测结果
    # 对于每张图，
    # 取检测的一个bbox bb
    # bb 和该图gt bbox bxes_gt计算 iou，去最大的iou

    ovmax = -np.inf
    tp, fp = 0, 0
    for bb in bbxes_pred:
        if len(BBGT) <= 0: 
            continue
        # 计算IoU
        cls_pred = bb[-1]
        BBGT_catg = BBGT[BBGT[:, -1]==cls_pred]
        if len(BBGT_catg) <=0:
            continue
        overlaps = get_iou(BBGT_catg, bb)
        ovmax = np.max(overlaps)   
        jmax = np.argmax(overlaps)    # 取最大的IoU的bbox索引
        if ovmax > ovthresh:  # 是否大于阈值
            tp += 1
#           删除已经匹配的检测框
            BBGT = np.concatenate([BBGT[~(BBGT[:, -1]==cls_pred)], 
                                   np.delete(BBGT_catg, jmax, axis = 0)], 
                                  axis = 0)
        else:
            fp += 1.
     
    return tp, fp

### 数据展示

In [ ]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/dada_det

In [ ]:
ROOT = Path('/home/chenyi/workspace/dataset/data/dada_det/hard_sample_0610/')

#### 处理打标标签

##### 构建中文标签和业务id映射

In [ ]:
# counter = df.value_counts(df.columns[3])
# name_list = [item[0] for item in list(counter.index)]
# name_df = pd.DataFrame()
# name_df['name'] = name_list

In [ ]:
import json
with open('../../../data/dict/name_chi2l1.json', 'r') as f:
    name_chi2l1 = json.load(f)

In [ ]:
df['label6'] = df[df.columns[3]].map(name_chi2l1)

In [ ]:
df.value_counts('label6')

In [ ]:
# 增加多出的中文标签映射
# name_chi2l1['皮衣外套'] = 4

In [ ]:
import json
with open('../../../data/dict/name_chi2l1.json', 'w') as f:
    json.dump(name_chi2l1, f)

In [ ]:
model_label_map

In [ ]:
l1_map_model = get_list2dict(model_label_map.values(), model_label_map.keys())
df['label_model'] = df['label6'].map(l1_map_model)

##### 存储目标index

In [ ]:
# df = pd.read_csv('../data/wh.csv')

In [ ]:
!ls /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
!mkdir -p /nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed')

In [ ]:
df.head(2).values

In [ ]:
df['img_tp'] = df['img_sp'].apply(lambda x:'{0}/{1}/{2}'.format(draw_root, Path(x).parts[-2], Path(x).name))

In [ ]:
sys.path.append('../../../')
from utils.plots.plots import draw_bbox
import cv2
from PIL import Image

In [ ]:
# error_p_list = []
# for ind, row in tqdm(df.iterrows()):
#     row_info = row.loc[df.columns[:8]].values
#     w,h = row_info[1], row_info[2]
#     name = row.loc[['label_model']].values[0]
#     bbox = list(row_info[4:8])
#     bbox.insert(0,name)
#     tp = Path(row.loc[['img_tp']].values[0])
#     if not tp.parent.is_dir():
#         tp.parent.mkdir()
#     rp = tp if tp.is_file() else row.loc[['img_sp']].values[0]
#     img = cv2.imread(str(rp))
#     img = draw_bbox(img, bbox)
#     try:
#         cv2.imwrite(str(tp), img)
#     except:
#         error_p_list.append(tp)
#     # break

In [ ]:
error_p_list

In [ ]:
error_df = pd.DataFrame()
error_df['path'] = error_p_list
error_df['stem'] = error_df['path'].apply(lambda x: Path(x).stem)

In [ ]:
df.head()

In [ ]:
df = df[~df['stem'].isin(error_df['stem'])]

In [ ]:
# df.to_csv('../data/wholebody5k0610/wholebody5k_july.csv', index=False)

#### Display Html

In [ ]:
sys.path.append('../../../')
from utils.display.html import *
from IPython.display import HTML
from io import BytesIO

In [ ]:
draw_root = Path('/nas/chenyi/datasets_cls/dada_det_drawed/wholebody_jul_drawed/')

In [ ]:
img_list = glob.glob(str(draw_root/'*/*.jpg'))
len(img_list), img_list[-1]

In [ ]:
# img_list = glob.glob(draw_root + '*.jpg')
df_drawed = pd.DataFrame()
df_drawed['path'] = img_list[-100:]

In [ ]:
size = 500
df_drawed['res'] = df_drawed['path'].map(lambda x: get_thumbnail(x, size))

In [ ]:
f = df_drawed[['res', 'path']].to_html(formatters={'res': image_formatter}, escape=False)

In [ ]:
HTML(f)

### 解析json文件

In [ ]:
json_list = glob.glob('/home/chenyi/workspace/dataset/haowei34k_labelImg/xml_label/*/*.json')

In [ ]:
json_df = pd.DataFrame()
json_df['spath'] = json_list

In [ ]:
xml_dir = Path('/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/')

In [ ]:
json_df['tpath'] = json_df['spath'].apply(lambda x:
                                         '{0}/{1}'.format(xml_dir, re.findall(r'/[0-9]{1,3}/(.*?.json)',x)[0]))

In [ ]:
sys.path.append('/home/chenyi/workspace/myPyModule99/data_format/json_to_xml')
from json_to_xml.main import *

In [ ]:
root_json_dir = xml_dir
root_save_xml_dir = '/home/chenyi/workspace/dataset/haowei34k_labelImg/json2xml/xml'

In [ ]:
import json
for json_filename in tqdm(os.listdir(root_json_dir)):
    json_path = os.path.join(root_json_dir, json_filename)
    json_data = json.load(open(json_path))
    break

In [ ]:
json_df['path'] = glob.glob(str(root_json_dir/'*.json'))

In [ ]:
json_df['fname'] =  json_df['path'].apply(lambda x: re.findall(r'/json2xml/(.*?).json',x)[0])

In [ ]:
json_labels = {}
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    json_labels[fname] = ''
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        if name not in classes_name.keys():
            classes_name[name] = len(classes_name)
        json_labels[fname] += '{} {:6f} {:6f} {:6f} {:6f}\n'.format(
            classes_name[name], xmin, ymin, xmax, ymax)

In [ ]:
fname_list = []
xmin_list = []
ymin_list = []
xmax_list = []
ymax_list = []
labels_list = []
for ind,row in tqdm(json_df.iterrows()):
    json_path = row['path']
    fname = row['fname']
    json_data = json.load(open(json_path))
    for item in json_data['shapes']:
        name = item['label']
        points = item['points']
        xmin, ymin = points[0]
        xmax, ymax = points[1]
        
        fname_list.append(fname)
        labels_list.append(name)
        xmin_list.append(xmin)
        ymin_list.append(ymin)
        ymax_list.append(ymax)
        xmax_list.append(xmax)

In [ ]:
json_bboxes = pd.DataFrame()

In [ ]:
json_bboxes['fname'] = fname_list
json_bboxes['label'] = labels_list
json_bboxes['xmin'] = xmin_list
json_bboxes['ymin'] = ymin_list
json_bboxes['xmax'] = xmax_list
json_bboxes['ymax'] = ymax_list

In [ ]:
json_bboxes.head()